In [ ]:
pip install fashion-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:0

In [ ]:
pip install Pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 11.7 MB/s eta 0:00:00


In [ ]:
from fashion_clip.fashion_clip import FashionCLIP

In [ ]:
import numpy as np
import torch
import os
from dotenv import load_dotenv
from pinecone import Pinecone
import gc

In [ ]:
# Load the FashionCLIP model
fclip = FashionCLIP("fashion-clip")

In [ ]:
# Pinecone 연결
load_dotenv()
api_key = os.getenv("API_KEY")
pc = Pinecone(api_key=api_key, environment="us-east-1")
tag_index = pc.Index("closhare-tags")  # 태그용 인덱스

In [ ]:
# Fashion-related tags for similarity matching
fashion_tags = {
    "Clothing Types": ["T-shirt", "Hoodie", "Sweatshirt", "Jacket", "Coat", "Blazer", "Vest", "Sweater", "Cardigan",
                       "Jeans", "Shorts", "Skirt", "Dress", "Jumpsuit", "Overalls",
                       "Leggings", "Trousers", "Pants", "Shirt", "Blouse", "Tank top", "Crop top",
                       "Belt", "Scarf", "Tie", "Hat", "Cap", "Beanie", "Gloves", "Socks", "Tights",
                       "Underwear", "Lingerie", "Pajamas", "Sleepwear", "Swimwear", "Bikini", "Cover-up", "Rash guard",
                       "Sneakers", "Running shoes", "Loafers", "Sandals", "Boots", "Heels",
                       "Backpack", "Tote bag", "Clutch bag", "Crossbody bag", "Wallet", "Sunglasses"],
    "Fashion Styles": ["Casual", "Streetwear", "Minimal", "Sporty", "Formal", "Business casual", "Chic", "Bohemian",
                       "Vintage", "Retro", "Preppy", "Grunge", "Gothic", "Punk", "Athleisure",
                       "Classic", "Eclectic", "Artsy", "Edgy", "Romantic", "Feminine",
                       "Masculine", "Androgynous", "Avant-garde", "Luxury", "High fashion", "Urban", "Cozy", "Relaxed",
                       "Sophisticated", "Elegant", "Playful", "Bold", "Subdued", "Timeless", "Transitional",
                          "Resort", "Cruise", "Festival", "Beachwear", "Loungewear", "Activewear", "Athletic",
                          "Outdoor", "Travel", "Weekend", "Date night", "Cocktail", "Evening", "Party", "Bridal",
                          "Gala", "Red carpet", "Black tie", "Business formal", "Smart casual"],
    "Patterns & Prints": ["Solid", "Striped", "Plaid", "Polka dot", "Floral", "Geometric", "Animal print",
                          "Camouflage", "Tie-dye", "Abstract", "Graphic", "Textured", "Color-blocked",
                          "Checked", "Houndstooth", "Paisley", "Chevron", "Argyle", "Damask",
                          "Ikat", "Brocade", "Jacquard", "Tartan", "Gingham", "Tye-dye", "Ombre", "Marble",
                            "Watercolor", "Metallic", "Glitter", "Foil", "Lace", "Mesh", "Sheer", "Transparent",
                            "Ribbed", "Quilted", "Puff print", "Satin", "Velvet", "Corduroy", "Canvas",
                            "Leatherette", "Faux fur", "Tulle", "Chiffon", "Organza", "Silk", "Denim",
                            "Taffeta", "Bamboo", "Linen", "Jersey", "Sweatshirt fabric", "Fleece", "Terry cloth",
                            "Neoprene", "Spandex", "Nylon", "Polyester", "Acrylic", "Rayon", "Viscose" ],
    "Materials": ["Cotton", "Denim", "Leather", "Silk", "Wool", "Cashmere", "Linen", "Polyester", "Nylon",
                  "Acrylic", "Rayon", "Viscose", "Spandex", "Bamboo", "Tencel", "Modal", "Faux fur",
                  "Satin", "Velvet", "Corduroy", "Canvas", "Taffeta", "Chiffon", "Organza",
                  "Jersey", "Sweatshirt fabric", "Fleece", "Terry cloth", "Neoprene", "Mesh", "Sheer",
                  "Transparent", "Ribbed", "Quilted", "Puff print", "Brocade", "Jacquard", "Tartan" ],
    "Colors": ["Black", "White", "Gray", "Beige", "Red", "Blue", "Green", "Yellow", "Pink", "Purple", "Brown",
               "Orange", "Turquoise", "Navy", "Olive", "Burgundy", "Teal", "Coral", "Lavender", "Mustard", "Cream",
                "Maroon", "Charcoal", "Ivory", "Peach", "Mint", "Aqua", "Magenta", "Cyan", "Indigo", "Fuchsia",
                "Emerald", "Sapphire", "Ruby", "Amber", "Amber", "Copper", "Bronze", "Silver", "Gold", "Platinum",
                "Rose gold", "Champagne", "Slate", "Dusty rose", "Seafoam", "Lilac", "Berry", "Rust", "Terracotta" ],
    "Seasons": ["Spring", "Summer", "Fall", "Winter"],
}

In [ ]:
# with torch.no_grad():
#     text_vec = fclip.encode_text(["A photo of jacket"], batch_size=1)[0]
# print("Text vector shape:", text_vec.shape)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

100%|██████████| 1/1 [00:00<00:00, 17.66it/s]

Text vector shape: (512,)


In [ ]:
# from PIL import Image

# image_path = "1.jpg"
# image = Image.open(image_path).convert("RGB")
# with torch.no_grad():
#     image_vec = fclip.encode_images([image], batch_size=1)[0]
# print("Image vector shape:", image_vec.shape)


100%|██████████| 1/1 [00:00<00:00,  7.67it/s]

Image vector shape: (512,)


In [ ]:
from more_itertools import chunked  # pip install more-itertools

all_texts = []
id_metadata_pairs = []

for category, category_tags in fashion_tags.items():
    for tag in category_tags:
        all_texts.append(f"A photo of {tag}")
        id_metadata_pairs.append((tag, category))

batch_size = 32  # 시스템에 맞게 조정

for texts_chunk, meta_chunk in zip(chunked(all_texts, batch_size), chunked(id_metadata_pairs, batch_size)):
    with torch.no_grad():
        vectors = fclip.encode_text(texts_chunk, batch_size=batch_size)
        vectors /= np.linalg.norm(vectors, axis=1, keepdims=True)

    upload_batch = []
    for (tag, category), vector in zip(meta_chunk, vectors):
        upload_batch.append({
            "id": tag,
            "values": vector.tolist(),
            "metadata": {
                "tag": tag,
                "category": category
            }
        })

    tag_index.upsert(upload_batch)
    print(f"[UPLOADED] {len(upload_batch)} tags")

In [ ]:
import os
from PIL import Image
from io import BytesIO

# Load and encode image
image_path = "3.jpg"
image = Image.open(image_path).convert("RGB")
with torch.no_grad():
    vector = fclip.encode_images([image], batch_size=1)[0]
    vector /= np.linalg.norm(vector)

# Define categories (same keys as fashion_tags)
categories = [
    "Clothing Types", "Fashion Styles",
    "Materials", "Colors", "Seasons"
]

# Search top tag per category
results = []

for category in categories:
    query_result = tag_index.query(
        vector=vector.tolist(),
        top_k=1,
        include_metadata=True,
        filter={"category": {"$eq": category}}  # Only search in this category
    )
    matches = query_result.get("matches", [])
    if matches:
        match = matches[0]
        results.append({
            "category": category,
            "tag": match["metadata"]["tag"],
            "score": match["score"]
        })
    else:
        results.append({
            "category": category,
            "tag": None,
            "score": None
        })

# Print results
for r in results:
    print(f"[{r['category']}] → {r['tag']} (score: {r['score']:.4f})" if r['tag'] else f"[{r['category']}] → No match")

100%|██████████| 1/1 [00:00<00:00, 32.35it/s]


[Clothing Types] → T-shirt (score: 0.2729)
[Fashion Styles] → Vintage (score: 0.2390)
[Materials] → Puff print (score: 0.2340)
[Colors] → Ivory (score: 0.2403)
[Seasons] → Summer (score: 0.1910)
